# Tutorial

Nangs is a python module built on top of pytorch to solve partial differential equations (PDEs).

## How does it works ?

Let's assume we want to solve the following PDE:

\begin{equation}
    \frac{\partial \phi}{\partial t} + u \frac{\partial \phi}{\partial x} = 0
\end{equation}

Different numerical techniques that solve this problem exist, and all of them are based on finding an approximate function that satisfies the PDE. Traditional numerical methods discretize the domain into small elements where a form of the solutions is assumed (for example, a constant) and then the final solution is composed as a piece-wise, discontinuous function.

Nangs uses the property of neural networks (NNs) as universal function approximators to find a continuous and derivable solution to the PDE, that requires significant less computing resources compared with traditional techniques and with the advantage of including the free-parameters as part of the solution.

The independen variables (i.e, $x$ and $t$) are used as input values for the NN, and the solution (i.e. $\phi$) is the output. In order to find the solution, at each step the NN outputs are derived w.r.t the inputs. Then, a loss function that matches the PDE is built and the weights are optimized accordingly. If the loss function goes to zero, we can assume that our NN is indeed the solution to our PDE.

We will see how to do that in this basic tutorial.

## Quick start

Here you can find the code required to solve the above mentioned PDE.

In [3]:
# import nangs
from nangs import PDE

# define custom PDE
class MyPDE(PDE):
    def __init__(inputs, outputs):
        super(inputs, outputs)
    def computePdeLoss(grads, params): 
        # here is where the magic happens
        dpdt, dpdx = grads['p']['t'], grads['p']['x']
        u = params['u']
        return dpdt + u*dpdx = 0

# instanciate the PDE with inputs, outputs and parameters
pde = MyPDE(inputs=['t','x','u'], outputs='p', params='u')
#pde = MyPDE(inputs=['t','x','u'], outputs='p', params='u')

# set input values
x = [0, 0.1, ..., 1.0]
t = [0, 0.1, ..., 1.0]
pde.setInputs({'x': x, 't': t})

# set free-parameters
u = 1
pde.addParam('u', u) 

# set boundary conditions
# periodic b.c for the space dimension
xb = [0, 1]
pde.addBoco({'x': xb, 't': t}, type='PERIODIC')
# initial condition (dirichlet for temporal dimension)
p0 = np.sin(2*pi*x)
pde.addBoco({'x': x, 't': 0, 'p': p0}, type='DIRICHLET')

# define solution topology
topo = {'layers': 10, 'neurons': 256, 'activations': ‘relu’}
pde.setTopology(topo)

# set optimization parameters
pde.setSolverParams(criterion, optimizer, …)

# find the solution
pde.solve() 

# evaluate the solution
x, t, u = [0, 0.1, …, 0.9, 1.0], 1, 1
p = pde.eval({'x': x, 't': t, 'u': 1})

# plt.plot(x, p)
# plt.show()
    


SyntaxError: invalid character in identifier (<ipython-input-3-c9a5ab587f0c>, line 10)

## Step by step guide

Let's go through the code step by step. First, we import the nangs module to acces its predefined classes and operations to solve PDEs with NNs.

In [13]:
from nangs import PDE

ModuleNotFoundError: No module named 'nangs'

### Defining the PDE

Then, we have to instanciate a PDE in order to solve it. We can use a predefined PDE or build our custom one. In this tutorial we are going to solve de one-dimensional advection equation with a custom implementation.

In [5]:
class MyPDE(PDE):
    def __init__(inputs, outputs):
        super(inputs, outputs)
    def computePdeLoss(grads, params): 
        # here is where the magic happens
        dpdt, dpdx = grads['p']['t'], grads['p']['x']
        u = params['u']
        return dpdt + u*dpdx = 0

SyntaxError: invalid syntax (<ipython-input-5-096b9beeae8f>, line 8)

When defining a custom PDE we are required to overwrite the **computePdeLoss** methods. This is the function that will be called during the optimization process, and we must ensure that the value returned matches our PDE. If not, we are going to solve a wrong equation.

We build the loss function by combining the derivatives of the NN ouputs w.r.t the inputs and free-parameters in the correct way. Notice that we use characters to retrieve the values ('p' for the output, 'x' and 't' for the inputs and 'u' for the parameter). We must ensure that this keys are created accordingly when the PDE is instanciated.

In [ ]:
pde = MyPDE(inputs=['t','x'], outputs='p', params='u')

The following lines of code have the same effect than the previous one.

In [7]:
# instanciate a PDE without parameters
pde = MyPDE()
# add inputs one by one or together with an array
pde.addInput('x')
pde.addInputs(['x','t'])
# add outputs one by one or together with an array
pde.addOutput('p')
pde.addOutputs(['p1','p2']) # if that is the case
# add parameters one by one or together with an array
pde.addParam('u')
pde.addParams(['u','v']) # if that is the case

NameError: name 'MyPDE' is not defined

Note that a free-parameter can also be included as input to the NN.

In [8]:
# short version
pde = MyPDE(inputs=['t','x','u'], outputs='p', params='u')

# long version
pde = MyPDE()
pde.addInputs(['x','t','u'])
pde.addOutput('p')
pde.addParam('u')

NameError: name 'MyPDE' is not defined

Different errors may arise when trying to define several inputs/outputs/parameters with the same key (depending on the order of definition). It is required a unique key for every inputs/outputs/parameter.

In [ ]:
pde = MyPDE(inputs=['t','x'], outputs='p', params='u')
pde.addInput('x') # error: x already exists
pde.addInput('p') # error: p already exists
pde.addParam('u') # error: u already exists
pde.addInput('u') # allowed since a parameter can also be input

At this point we only have declared keys to our variables. In order to solve the PDE we have to specify actual values for all this variables that will be evaluated during the optimization process.

In [9]:
# set input values
x = [0, 0.1, ..., 1.0]
t = [0, 0.1, ..., 1.0]
pde.setInputs({'x': x, 't': t})

# set free-parameters
u = 1
pde.addParam('u', u) 

NameError: name 'pde' is not defined

We can also add inputs one by one and several parameters at once.

In [11]:
x = [0, 0.1, ..., 1.0]
pde.setInput('x', x)

t = [0, 0.1, ..., 1.0]
pde.setInput('t', t)

u, v = 1, 2
pde.addParam({'u': u, 'v': v}) 

NameError: name 'pde' is not defined

In order to be able to solve the PDE you will need to specify at least one value for each input/parameter.

### Defining boundary conditions

We could attempt to start finding the solution at this point, and probably we would find one. Nevertheless that would be a trivial solution since a PDE is constrained by boundary conditions. We can add them as follows.

In [12]:
# periodic b.c for the space dimension
xb = [(0, 1)]
pde.addBoco({'x': xb, 't': t}, type='PERIODIC')

# initial condition (dirichlet for temporal dimension)
p0 = np.sin(2*pi*x)
pde.addBoco({'x': x, 't': 0, 'p': p0}, type='DIRICHLET')

NameError: name 'pde' is not defined

Boundary conditions must be specified one at a time. Depending on the **type** a different number of parameters are required. 

A PERIODIC boundary conditions requires pairs of values that will be enforced to be equal at every step using a mean squared optimization.

A DIRICHLET boundary condition requires a specific value for the outputs that will be matched using a typical mean squared optimization.

You can see a list of boundary conditions here.

### NN topology and optimization parameters

We encode the solution to the PDE using a multi-layer perceptron (MLP).

There are two ways to define the topoly.

In [ ]:
# compact form
topo = {'layers': 10, 'neurons': 256, 'activations': 'relu'}

# layer by layer
topo = [{'layers': 1, 'neurons': 256, 'activations': 'relu'},
        {'layers': 2, 'neurons': 512, 'activations': 'relu'},
        {'layers': 1, 'neurons': 32, 'activations': 'sigmoid'}]


pde.setTopology(topo)

Then, we define our optimization criteria and optimizer. More on this on Pytorch reference guide.

In [ ]:
# set optimization parameters
pde.setSolverParams(criterion, optimizer, …)

### Solving the PDE

With everything in place, we can now solve the PDE. 

In [15]:
pde.solve()

NameError: name 'pde' is not defined

We can get a summary of the problem

In [17]:
pde.summary() 

NameError: name 'pde' is not defined

Depending on the problem configuration this will take more or less time. If the loss function does not goes to zero, something is wrong. 

### Evaluate the solution

Once the training is done, we can evaluate our solution.

In [18]:
x, t, u = [0, 0.1, …, 0.9, 1.0], 1, 1
p = pde.eval({'x': x, 't': t, 'u': 1})

# plt.plot(x, p)
# plt.show()

SyntaxError: invalid character in identifier (<ipython-input-18-62246d6f3dc6>, line 1)

We can pass an array of values for each input, getting the results accordingly.

In [ ]:
x, t, u = [0, 0.1, …, 0.9, 1.0], [0, 0.1, 0.2], 1
p = pde.eval({'x': x, 't': t, 'u': 1})
# p = [[p0t0, p1t0, … ],[p0t1, p1t1, … ],[p0t2, p1t2, … ]]

### Saving and loading the model

We can save the solution using standard pytorch notation. The NN can be accesed as

In [19]:
model = pde.solution()
PATH = 'MyPDE_solution.pt'

# save model
torch.save(model.state_dict(), PATH)

NameError: name 'pde' is not defined

Then we can load the model to continue training or just use it to get a solution. Be sure to call the **init** function to build the internal model before loading the weights. When loading a model for evaluation, it is not required to set values for 

In [20]:
pde.init()

model.load_state_dict(torch.load(PATH))

# to continue training, set model in training mode
model.train()

# to evaluate, set model in eval mode
model.eval()

NameError: name 'model' is not defined

### Author

Juan Sensio - juansensio03@gmail.com